<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Mini - Projet
# Classification de dépêches d’agence avec NLTK

**Objectifs**

L’objectif de ce projet est de réaliser des expériences de *classification de documents* sous NLTK avec 
le corpus de dépêches Reuters. 

*  le corpus Reuters contient environ 10'000 dépêches datant des années 1980, et il est fourni avec NLTK comme expliqué dans le [livre NLTK, ch.2](http://www.nltk.org/book/ch02.html), §1.4.

In [2]:
import nltk
from nltk.corpus import reuters
nltk.downloader.Downloader().download('reuters') 
# à exécuter une seule fois pour télécharger les fichiers localement

[nltk_data] Downloading package reuters to C:\Users\Lenovo
[nltk_data]     T50s\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

* Le code suivant illustre certaines de ses fonctionnalités en imprimant des informations de base pour la collection.

In [59]:
# List of documents
documents = reuters.fileids()
print(str(len(documents)) + " documents");
 
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
documents));
print(str(len(train_docs)) + " total train documents");
 
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
documents));
print(str(len(test_docs)) + " total test documents");
 
# List of categories
categories = reuters.categories();
print(str(len(categories)) + " categories");
 
# Documents in a category
category_docs = reuters.fileids("interest");
 
# Words for a document
document_id = category_docs[0]
document_words = reuters.words(category_docs[0]);
print(document_words);
 
# Raw document
print(reuters.raw(document_id));

10788 documents
7769 total train documents
3019 total test documents
90 categories
['BUNDESBANK', 'ALLOCATES', '6', '.', '1', 'BILLION', ...]
BUNDESBANK ALLOCATES 6.1 BILLION MARKS IN TENDER
  The Bundesbank accepted bids for 6.1
  billion marks at today's tender for a 28-day securities
  repurchase pact at a fixed rate of 3.80 pct, a central bank
  spokesman said.
      Banks, which bid for a total 12.2 billion marks liquidity,
  will be credited with the funds allocated today and must buy
  back securities pledged on May 6.
      Some 14.9 billion marks will drain from the market today as
  an earlier pact expires, so the Bundesbank is effectively
  withdrawing a net 8.1 billion marks from the market with
  today's allocation.
      A Bundesbank spokesman said in answer to enquiries that the
  withdrawal of funds did not reflect a tightening of credit
  policy, but was to be seen in the context of plentiful
  liquidity in the banking system.
      Banks held an average 59.3 billion m

# Hyper-paramètres 

Avant d’appliquer les techniques traditionnelles d’apprentissage automatique, nous devons représenter et pondérer chaque document par rapport à l’ensemble des fonctionnalités textuelles. Nous allons appliquer les transformations suivantes :

* Minuscules le contenu d’origine
* Tokeniser le texte
* Stem chacun des jetons
* Pondérer chacune de ces caractéristiques, pour chaque document
* Normaliser la représentation

In [80]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

from nltk.corpus import stopwords
stopWords = stopwords.words('english')
charfilter = re.compile('[a-zA-Z]+')

def simple_tokenizer(text):
    #tokenizing the words:
    #words = word_tokenize(text)
    #converting all the tokens to lower case:
    words = map(lambda word: word.lower(), text)
    #let's remove every stopwords
    words = [word for word in words if word not in stopWords]
    #stemming all the tokens
    tokens = (list(map(lambda token: PorterStemmer().stem(token), words)))
    ntokens = list(filter(lambda token : charfilter.match(token),tokens))
    return ntokens

* nous pouvons créer des classifieurs qui baliseront automatiquement les nouveaux documents avec des étiquettes de catégorie appropriées. Tout d’abord, nous construisons une liste de documents, étiquetés avec les catégories appropriées.

In [81]:
from nltk.corpus import reuters
import random

list_categorie = ["money-fx", "interest", "money-supply"]
document = [(list(reuters.words(reuterid)), category)
             for category in list_categorie
             for reuterid in reuters.fileids(category)]
random.shuffle(document)


In [82]:
def document_features(document): 
    document_words = set(document)
    word_features= simple_tokenizer(document_words)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

# classifieurs binaires NaiveBayes


In [83]:
featuresets = [(document_features(d), c) for (d,c) in document]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [84]:
 print(nltk.classify.accuracy(classifier, test_set))

0.69


In [85]:
sorted(classifier.labels())

['interest', 'money-fx', 'money-supply']

In [86]:
classifier.show_most_informative_features(5)

Most Informative Features
           contains(mln) = False          money- : intere =     60.5 : 1.0
           contains(feb) = False          money- : money- =     56.7 : 1.0
         contains(defin) = False          money- : money- =     48.4 : 1.0
            contains(m3) = False          money- : money- =     37.3 : 1.0
           contains(plu) = False          money- : intere =     36.4 : 1.0


In [88]:
Y_test = classifier.predict(test_set)
print(classification_report(Y_test, test_labels))

AttributeError: 'NaiveBayesClassifier' object has no attribute 'predict'

# Classifieur d’arbre de décisions

In [76]:
from nltk.corpus import reuters
import random

list_categorie = [ "grain", "wheat", "corn"]
document_arbre = [(list(reuters.words(reuterid)), category)
             for category in list_categorie
             for reuterid in reuters.fileids(category)]
random.shuffle(document_arbre)

In [79]:
featuresets = [(document_features(d), c) for (d,c) in document_arbre]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.classify.DecisionTreeClassifier.train(
    train_set, entropy_cutoff=0, support_cutoff=0)

KeyboardInterrupt: 

In [ ]:
 print(nltk.classify.accuracy(classifier, test_set))